##### O intuito do código é analisar dados por grupo vulnerável de acordo com as categorias do ISP-RJ. Para isso, pivotei 5 colunas com valores 0 e 1 com 0 para o caso que não se encaixe no grupo e 1 para os que se encaixam. O objetivo de pivotar as colunas é evitar repetições de linhas, visto que um caso pode fazer parte de mais de um grupo.
###### Atenção: As condições para classificação de cada grupo estão disponíveis no site do ISP-RJ.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import psycopg2 as pg
import json
import locale
import requests
from datetime import date
from sqlalchemy import create_engine

##### Conexão ao banco e extração dos dados lendo a query de um arquivo sql.

In [ ]:
# Parâmetros de conexão
conn_parameters = {
        'host': 'host',
        'database': 'banco',
        'user': 'user',
        'password': 'senha'
    }

try:
    engine = create_engine(f"postgresql+psycopg2://{conn_parameters['user']}:{conn_parameters['password']}@{conn_parameters['host']}/{conn_parameters['database']}")
    print("Conexão estabelecida")

    # Leitura do arquivo sql
    with open('consulta.sql', 'r') as file:
        query_sql = file.read()

    # Extração dos dados em um dataframe
    df = pd.read_sql_query(query_sql, engine)
    display(df.head(3))

    print("Extração de dados concluída.")
    print("Conexão fechada.")

except Exception as error:
    print(f"Erro ao fazer a extração de dados do banco: {error}")


Conexão estabelecida


,controle,ano,mes,ano_fato,mes_fato,titulo,titulo_do,conteudo,data_com,data_fato,idade,municipio_fato,sexo,cor,relacao,Crianças e Adolescentes,jovens,idosos,População Negra,mulheres
0,00787401-2020,2020,Janeiro,2020,Janeiro,"Lesão Corporal Provocada por Socos, Tapas e Po...",Lesão corporal dolosa,vítimas,2020-01-01,2020-01-01,44,Rio de Janeiro (Capital),feminino,parda,Outra,0,0,0,1,1
1,00987771-2020,2020,Janeiro,2020,Janeiro,Homicídio Provocado por Projétil de Arma de Fogo,Homicídio doloso,vítimas,2020-01-01,2020-01-01,36,Macaé,masculino,parda,Nenhuma,0,0,0,1,0
2,00987771-2020,2020,Janeiro,2020,Janeiro,Homicídio Provocado por Projétil de Arma de Fogo,Homicídio doloso,vítimas,2020-01-01,2020-01-01,49,Macaé,feminino,ignorado,Nenhuma,0,0,0,0,1


Extração de dados concluída.
Conexão fechada.


##### Modelagem dos dados para o primeiro e segundo gráfico

In [ ]:
# Colunas que vão ser filtradas
columns = ['municipio_fato', 'Crianças e Adolescentes', 'mulheres', 'jovens', 'idosos', 'População Negra']

# Filtrando apenas as colunas de interesse do DataFrame
df_filter = df[columns]

# Agrupando por 'municipio_fato' e calculando a soma para cada grupo
df_map = df_filter.groupby('municipio_fato').sum().reset_index()

# Como quero uma coluna com a quantidade de casos por municipio, é necessário criar ela a parte e depois fazer o join como dataframe que será utilizado
casos = df.groupby('municipio_fato').size().reset_index(name = 'Casos')
display(casos)

# Join
df_map = df_map.merge(casos, on = 'municipio_fato', how = 'left')

# Removendo '(Capital)' de 'Rio de Janeiro' para ficar igual ao registro do IBGE e evitar erros
df_map['municipio_fato'] = df_map['municipio_fato'].replace('Rio de Janeiro (Capital)', 'Rio de Janeiro')

# Usando .idxmax(axis=1) para encontrar o nome da coluna com o valor máximo em cada linha
grupos = ['Crianças e Adolescentes', 'mulheres', 'jovens', 'idosos', 'População Negra']
df_map['Grupo'] = df_map[grupos].idxmax(axis=1)

display(df_map)

,municipio_fato,Casos
0,Angra dos Reis,8235
1,Aperibé,493
2,Araruama,7369
3,Areal,488
4,Armação dos Búzios,3206
...,...,...
87,Três Rios,5283
88,Valença,3712
89,Varre-Sai,353
90,Vassouras,1732


,municipio_fato,Crianças e Adolescentes,mulheres,jovens,idosos,População Negra,Casos,Grupo
0,Angra dos Reis,362,3500,1797,3072,2908,8235,mulheres
1,Aperibé,24,287,133,88,168,493,mulheres
2,Araruama,306,2938,1473,2755,3129,7369,População Negra
3,Areal,47,259,128,126,192,488,mulheres
4,Armação dos Búzios,119,1304,657,1121,988,3206,mulheres
...,...,...,...,...,...,...,...,...
87,Três Rios,338,2581,1310,1574,2343,5283,mulheres
88,Valença,196,1742,960,957,1575,3712,mulheres
89,Varre-Sai,31,185,109,61,121,353,mulheres
90,Vassouras,114,825,440,475,775,1732,mulheres


##### Coleta do geojson a partir de uma API.

In [ ]:
url_malhas = 'https://servicodados.ibge.gov.br/api/v3/malhas/estados/33?formato=application/vnd.geo+json&qualidade=intermediaria&intrarregiao=municipio'
malhas_response = requests.get(url_malhas)
geojson_malhas = malhas_response.json()

url_municipios = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados/33/municipios?view=nivelado'
municipios_response = requests.get(url_municipios)
dados_municipios = municipios_response.json()

##### Adicionando o código IBGE ao dataframe

In [ ]:
# Criar um dicionário para mapear nomes de municípios para seus códigos
municipios_codigos = {
    municipio['municipio-nome']: municipio['municipio-id'] for municipio in dados_municipios
    }
print(municipios_codigos)

# Mapear os códigos do município no DataFrame usando o nome do município como chave
df_map['codigo_ibge'] = df_map['municipio_fato'].map(municipios_codigos)
display(df_map)

{'Angra dos Reis': 3300100, 'Aperibé': 3300159, 'Araruama': 3300209, 'Areal': 3300225, 'Armação dos Búzios': 3300233, 'Arraial do Cabo': 3300258, 'Barra do Piraí': 3300308, 'Barra Mansa': 3300407, 'Belford Roxo': 3300456, 'Bom Jardim': 3300506, 'Bom Jesus do Itabapoana': 3300605, 'Cabo Frio': 3300704, 'Cachoeiras de Macacu': 3300803, 'Cambuci': 3300902, 'Carapebus': 3300936, 'Comendador Levy Gasparian': 3300951, 'Campos dos Goytacazes': 3301009, 'Cantagalo': 3301108, 'Cardoso Moreira': 3301157, 'Carmo': 3301207, 'Casimiro de Abreu': 3301306, 'Conceição de Macabu': 3301405, 'Cordeiro': 3301504, 'Duas Barras': 3301603, 'Duque de Caxias': 3301702, 'Engenheiro Paulo de Frontin': 3301801, 'Guapimirim': 3301850, 'Iguaba Grande': 3301876, 'Itaboraí': 3301900, 'Itaguaí': 3302007, 'Italva': 3302056, 'Itaocara': 3302106, 'Itaperuna': 3302205, 'Itatiaia': 3302254, 'Japeri': 3302270, 'Laje do Muriaé': 3302304, 'Macaé': 3302403, 'Macuco': 3302452, 'Magé': 3302502, 'Mangaratiba': 3302601, 'Maricá': 

,municipio_fato,Crianças e Adolescentes,mulheres,jovens,idosos,População Negra,Casos,Grupo,codigo_ibge
0,Angra dos Reis,362,3500,1797,3072,2908,8235,mulheres,3300100
1,Aperibé,24,287,133,88,168,493,mulheres,3300159
2,Araruama,306,2938,1473,2755,3129,7369,População Negra,3300209
3,Areal,47,259,128,126,192,488,mulheres,3300225
4,Armação dos Búzios,119,1304,657,1121,988,3206,mulheres,3300233
...,...,...,...,...,...,...,...,...,...
87,Três Rios,338,2581,1310,1574,2343,5283,mulheres,3306008
88,Valença,196,1742,960,957,1575,3712,mulheres,3306107
89,Varre-Sai,31,185,109,61,121,353,mulheres,3306156
90,Vassouras,114,825,440,475,775,1732,mulheres,3306206


##### Adicionando o nome dos municípios ao geojson

In [ ]:
codigo_para_nome = {
    str(municipio['municipio-id']): municipio['municipio-nome'] for municipio in dados_municipios
                }
print(codigo_para_nome)

# Adicionar os nomes dos municípios ao GeoJSON
for feature in geojson_malhas['features']:
    codigo_municipio = feature['properties']['codarea']  # onde 'codarea'é o código do ibge na api das malhas
    feature['properties']['name'] = codigo_para_nome.get(codigo_municipio)

{'3300100': 'Angra dos Reis', '3300159': 'Aperibé', '3300209': 'Araruama', '3300225': 'Areal', '3300233': 'Armação dos Búzios', '3300258': 'Arraial do Cabo', '3300308': 'Barra do Piraí', '3300407': 'Barra Mansa', '3300456': 'Belford Roxo', '3300506': 'Bom Jardim', '3300605': 'Bom Jesus do Itabapoana', '3300704': 'Cabo Frio', '3300803': 'Cachoeiras de Macacu', '3300902': 'Cambuci', '3300936': 'Carapebus', '3300951': 'Comendador Levy Gasparian', '3301009': 'Campos dos Goytacazes', '3301108': 'Cantagalo', '3301157': 'Cardoso Moreira', '3301207': 'Carmo', '3301306': 'Casimiro de Abreu', '3301405': 'Conceição de Macabu', '3301504': 'Cordeiro', '3301603': 'Duas Barras', '3301702': 'Duque de Caxias', '3301801': 'Engenheiro Paulo de Frontin', '3301850': 'Guapimirim', '3301876': 'Iguaba Grande', '3301900': 'Itaboraí', '3302007': 'Itaguaí', '3302056': 'Italva', '3302106': 'Itaocara', '3302205': 'Itaperuna', '3302254': 'Itatiaia', '3302270': 'Japeri', '3302304': 'Laje do Muriaé', '3302403': 'Maca

##### Primeiro gráfico

In [ ]:
map1 = px.choropleth_mapbox(
    df_map, # df com os dados
    geojson = geojson_malhas, # geojson com os dados das malhas
    color = "Casos", # coluna do df com os valores para colorir o gráfico
    locations = "municipio_fato", # coluna do df utilizada como chave
    featureidkey = "properties.name", # valor do geojson utilizado como chave
    title = "Casos por município (Jan/2020 a Jun/2023)",
    mapbox_style = "white-bg", # estilo do mapa (não terá, optei por fundo branco)
    center={"lat": -22.1, "lon": -43}, # posição do mapa
    zoom = 7, # zoom inicial
    opacity = 0.7, # opacidade para que apareça o fundo (nesse caso como o fundo é branco, utilizei apenas para clarear um pouco as cores)
    height = 700 # comprimento do mapa
)
map1.show()

##### Segundo gráfico.

In [ ]:
map2 = px.choropleth_mapbox(
    df_map,
    geojson = geojson_malhas,
    color = "Grupo",
    locations = "codigo_ibge", # coluna do df utilizada como chave
    featureidkey = "properties.codarea", # valor do geojson utilizado como chave
    title = "Grupo vulnerável por municipio (Jan/2020 a Jun/2023)",
    mapbox_style = "white-bg",
    center={"lat": -22.1, "lon": -43},
    zoom = 7,
    opacity = 0.7,
    height = 700
)
map2.show()

##### Modelagem dos dados para o gráfico de barras.

In [ ]:
# Criando um novo dataframe transformando colunas em linhas e ordenando por casos em ordem decrescente.
df_grupo = df_filter.drop('municipio_fato', axis=1).sum().reset_index().rename(columns = {'index': 'Grupo', 0: 'Casos'}).sort_values(by = 'Casos', ascending = False)
display(df_grupo)

,Grupo,Casos
3,idosos,357269
1,mulheres,291743
4,População Negra,269759
2,jovens,156157
0,Crianças e Adolescentes,31905


##### Gráfico de barras.

In [ ]:
fig = px.bar(
    df_grupo, # df a ser utilizado
    x = 'Grupo', # valores do eixo x
    y = 'Casos', # valores do eixo y
    title = "Quantidade de casos por grupo vulnerável (Jan/2020 a Jun/2023)", 
    height = 600, # altura da figura
    color_discrete_sequence = ['#6A5ACD'] # escolhendo a cor roxo
    )
fig.update_layout(width = 800) # largura da imagem

fig.show()

##### Modelagem dos dados para os gráficos de linhas.

In [ ]:
import locale
locale.setlocale(locale.LC_TIME, 'pt_BR') # isso é para o pandas entender os meses em português certo, já que o padrão é em inglês.

columns = ['ano', 'mes', 'Crianças e Adolescentes', 'mulheres', 'jovens', 'idosos', 'População Negra']

# Filtrando o ano de 2020
df_filter2 = df[columns].loc[df['ano'] == 2020]
# Transformando os meses para numeral para ficar na ordem correta
df_filter2.loc[:, 'mes'] = pd.to_datetime(df_filter2['mes'], format='%B').dt.month
display(df_filter2)

# Agrupando por 'ano' e 'mes' e calculando a soma para cada grupo
df_line = df_filter2.groupby(['ano', 'mes']).sum().reset_index()
# Transformando os meses para a nomenclatura
df_line['mes'] = pd.to_datetime(df_line['mes'], format = '%m').dt.strftime('%B')

display(df_line)

,ano,mes,Crianças e Adolescentes,mulheres,jovens,idosos,População Negra
0,2020,1,0,1,0,0,1
1,2020,1,0,0,0,0,1
2,2020,1,0,1,0,0,0
3,2020,1,1,1,0,0,1
4,2020,1,0,0,1,0,1
...,...,...,...,...,...,...,...
177002,2020,12,0,0,0,1,0
177003,2020,12,0,1,1,0,0
177004,2020,12,0,1,0,0,1
177005,2020,12,0,0,0,1,0


,ano,mes,Crianças e Adolescentes,mulheres,jovens,idosos,População Negra
0,2020,Janeiro,873,8021,4511,5400,7382
1,2020,Fevereiro,836,7721,4379,4986,7073
2,2020,Março,675,5782,3230,4196,5338
3,2020,Abril,345,3822,2187,3463,3527
4,2020,Maio,338,3721,2048,4339,3419
5,2020,Junho,464,4822,2505,6176,4305
6,2020,Julho,568,6005,3228,7269,5482
7,2020,Agosto,668,6512,3544,5923,6034
8,2020,Setembro,712,6745,3797,5303,6224
9,2020,Outubro,703,6887,3803,5901,6510


##### Primeiro gráfico de linhas

In [ ]:
# Criando a figura
fig_line = go.Figure()

# Adicionando os traços de cada coluna
fig_line.add_trace(go.Scatter(x = df_line['mes'], y = df_line['Crianças e Adolescentes'], mode = 'lines', name = 'Crianças e Adolescentes'))
fig_line.add_trace(go.Scatter(x = df_line['mes'], y = df_line['mulheres'], mode = 'lines', name = 'Mulheres'))
fig_line.add_trace(go.Scatter(x = df_line['mes'], y = df_line['jovens'], mode = 'lines', name = 'Jovens'))
fig_line.add_trace(go.Scatter(x = df_line['mes'], y = df_line['idosos'], mode = 'lines', name = 'Idosos'))
fig_line.add_trace(go.Scatter(x = df_line['mes'], y = df_line['População Negra'], mode = 'lines', name = 'População Negra'))

# Ajustando o layout
fig_line.update_layout(
    title = 'Quantidade de registros por mês (2020)',
    xaxis_title = 'Casos',
    yaxis_title = 'Mês',
    plot_bgcolor = 'white'
    )

fig_line.update_xaxes(showline = True, linewidth = 1, linecolor = 'black')
fig_line.update_yaxes(showline = True, linewidth = 1, linecolor = 'black')

fig_line.show()

##### Dados para o segundo gráfico de linhas.

In [ ]:
columns2 = ['ano_fato', 'mes_fato', 'Crianças e Adolescentes', 'mulheres', 'jovens', 'idosos', 'População Negra']

df_filter3 = df[columns2].loc[df['ano_fato'] == '2020']
df_filter3.loc[:, 'mes_fato'] = pd.to_datetime(df_filter3['mes_fato'], format='%B').dt.month

df_line2 = df_filter3.groupby(['ano_fato', 'mes_fato']).sum().reset_index()
df_line2['mes_fato'] = pd.to_datetime(df_line2['mes_fato'], format = '%m').dt.strftime('%B')

##### Segundo gráfico de linhas.

In [ ]:
fig_line2 = go.Figure()

fig_line2.add_trace(go.Scatter(x = df_line['mes'], y = df_line['idosos'], mode = 'lines', name = 'Idosos (registro)')) # Linha da quantidade de registros por mês
fig_line2.add_trace(go.Scatter(x = df_line2['mes_fato'], y = df_line2['idosos'], mode = 'lines', name = 'Idosos (caso)')) # Linha da quantidade de ocorrência do fato por mês

fig_line2.update_layout(
    title = 'Quantidade de casos e registros por mês (2020)',
    xaxis_title = 'Casos',
    yaxis_title = 'Mês'
    )

fig_line2.show()